# Calculation of RSE and quantitative determination of PFAS concentrations from the LC-TQ V_2
Author: Valerie de Rijk

Date: 15/8/2025
Last updated: 8/1/2026

## 1 Data Loading
First we load in the raw data as a csv file (which is the file you directly get from MassHunter).
We also directly check if the s/n ratio is above 3 for all samples in the calibration line. 

In [ ]:
%run src/functions_quantitative2.ipynb
rawdata = load_and_label_pfas_csv("rawdata/Results_Test_SPE50ml_16-12-2025.csv")
sn_results = check_calibration_sn_ratios(rawdata)
# Print formatted results
print_sn_check_results(sn_results)

## 2 Calibration line:  Average Response Factor, recovery and RSE for EIS compounds 
To analyze all instrument linearity metrics, we first need to calculate the EIS concentrations found in the samples. 
We do this by caclulating the concentrations based on the formula on page 44 of EPA method 1633. 
To achieve this, we first need to calculate the average response factor per compound (EPA method 1633 page 28). 
Afterwards, we check if the recovery of the compound falls within the accepted limits denoted by EPA method 1633 (Table 6). 
We also already compute the concentrations for the QC samples, these are analyzed further in section 3.2 

We compute the RSE based on  the formula for RSE based on EPA method 1633 (section 10.3.3.3, page 28, option 2). The functions are loaded in a different script. 
If the RSE of a single compound remains above 20%, this means something is wrong with the linearity of the LC-MS TQ and the calculated concentrations in section 3 are not trustworthy. 

### 2.1 EIS compounds
MAKE SURE YOU CHANGE THE MATRIX TYPE FOR ACCURATE RECOVERIES

If you notice here certain calibration levels are consistently bad, there's an option to exclude them for further analysis in the 2nd code block. 
If you don't want to exclude any levels type *excluded_cal_level = 'None'*

The 2nd code block prints detailed information on the RSE. You can view detailed results by removing *%%capture* in the code block, or alternatively just observe the printed summary. 

### QC Cal samples 
If there's any QC samples included that have a level identified (i.e. CS4), the script will also print the results for this. If these recoveries do not meet required recovery standards, please proceed with caution.

In [ ]:
from expected_concs import expected_concs_EIS, expected_concs_target_analytes
from analogs_compounds import target_EIS_analogs, EIS_NIS_analogs
from solutions import calibration_solutions
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous,EIS_NIS_compound_recovery_solid, OPR_recovery_aqueous, OPR_recovery_solid, IPR_recovery_rsd_solid, IPR_recovery_rsd_aqueous

matrix_type = "aqueous"  # or "solid

# Select appropriate recovery limits based on matrix type
if matrix_type.lower() == "aqueous":
    EIS_NIS_compound_recovery = EIS_NIS_compound_recovery_aqueous
    OPR_recovery = OPR_recovery_aqueous
    IPR_recovery_rsd = IPR_recovery_rsd_aqueous
elif matrix_type.lower() == "solid":
    EIS_NIS_compound_recovery = EIS_NIS_compound_recovery_solid
    OPR_recovery = OPR_recovery_solid
    IPR_recovery_rsd = IPR_recovery_rsd_solid
else:
    raise ValueError(f"Unknown matrix type: {matrix_type}. Use 'aqueous' or 'solid'")

#calculations
eis_RFS = calculate_average_RFs_EIS(rawdata, EIS_NIS_analogs, calibration_solutions)


Ws_cal = 0.25#ml
dilution_NIS_stock_cal = 40 #dilution cal
dilution_EIS_stock_cal = 40 #dilution cal
spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount = 0.05 #ml

#calibration line
conc_EIS_cal, conc_QC = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)

calculated_recoveries_EIS_cal = calculate_recoveries(conc_EIS_cal, expected_concs_EIS)
calculated_recoveries_EIS_cal.to_csv("results/calculated_recoveries_EIS_cal.csv", index=False)
conc_EIS_cal, conc_QC = calculate_conc_EIS_cal(rawdata, eis_RFS, dilution_NIS_stock_cal, Ws_cal,
                          EIS_NIS_analogs, Df_cal, NIS_stock, spiked_amount_cal)
validation_results_cal = validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery)
validation_results_cal= validate_recoveries(calculated_recoveries_EIS_cal, EIS_NIS_compound_recovery)
summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")
failed_recoveries_cal = get_failed_recoveries(validation_results_cal)
# QC calculations - only run if QC samples exist
if conc_QC is not None and not conc_QC.empty:
    calculated_recoveries_QC_cal = calculate_recoveries(conc_QC, expected_concs_EIS)
    calculated_recoveries_QC_cal.to_csv("results/calculated_recoveries_QC_cal.csv", index=False)
    validation_results_QC = validate_recoveries(calculated_recoveries_QC_cal, EIS_NIS_compound_recovery)



In [ ]:
%%capture
# RSE EIS Calibration line
excluded_cal_level = 'L6'
rse_results_cal_EIS = calculate_eis_rse_modified(conc_EIS_cal, expected_concs_EIS, p=2, exclude_levels = excluded_cal_level)


In [ ]:
df_rse_eis=df_rse_results(rse_results_cal_EIS)
display(df_rse_eis)

## 2.2 Target analytes 

### 2.2.1 Response Ratio (RR) and Response Factor (RF) for target ID compounds
Here we compute the response ratio for the ID compounds by using the formula from EPA 1633 on page 27 under section 10.3.3.2. For now we also fix some naming errors in the raw data for the target analytes. The only difference with the above calculation is that EIS analogs instead of direct pairs are used for the target EIS compounds. For ease, we call both RF.

- *Note 1*: NMeFoSA and NEtFoSa are calculated now but are not included in the PFAC30PAR ampoule we currently use in the lab, so these compounds cannot be in the calibration line. Use these results with care.
- *Note 2*:EPA 1633 mentions that PFTrDA recovery should improve by taking the average of the analog EIS compounds 13c2-PFTeDA and 13c2-PFDoa. Currently it only takes the EIS compound 13C2-PFTeDA as an analog.
- *Note 3*: EPA 1633 is capable of analyzing 40 compounds, our native standard solution only contains 30, so you will get errors that data for some compounds is not found. This is correct. If new ampoules are ordered for the other 10 compounds, everything needed for the calculation is there.
- *Note 4*: the FTS compounds only have 6 cal levels instead of 7.


In [ ]:
from analogs_compounds import target_EIS_analogs

# Make a copy of rawdata to avoid modifying the original
rawdata_copy = rawdata.copy()
columns_to_rename = {}
for col in rawdata_copy.columns:
    new_col = col
    # Fix PFPHpA -> PFHpA
    if 'PFPHpA' in col:
        new_col = new_col.replace('PFPHpA', 'PFHpA')
    # Remove spaces from FTS compounds (e.g., "4:2 FTS" -> "4:2FTS")
    if ' FTS' in col:
        new_col = new_col.replace(' FTS', 'FTS')
    
    if new_col != col:
        columns_to_rename[col] = new_col

if columns_to_rename:
    rawdata_copy.rename(columns=columns_to_rename, inplace=True)

RF_target_analytes= calculate_average_RRs_targets(rawdata_copy, expected_concs_EIS, calibration_solutions, target_EIS_analogs)

### 2.2.2 Recovery IPR and OPR for target analytes in calibration line 
Here we compare calculated recoveries for the target analytes in all calibration samples and compare them to the expected recoveries and maximum RSD for IPR (Table 5) 
* Note: OPR/LLOPR recovery is a little less strict. If needed for comparison exchange IPR_recovery_rsd_aqueous for OPR_recovery_aqueous. IPR considers mean recovery and RSD, OPR is per sample. Thus, you should not run the function mean_recoveries. 

In [ ]:
from allowed_recoveries import IPR_recovery_rsd_aqueous, OPR_recovery_aqueous
sampletype = "Cal"
conc_target_analytes_cal = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
                           Df_cal, sampletype, debug_compound=None )
calculated_recoveries_target_cal = calculate_recoveries(conc_target_analytes_cal, expected_concs_target_analytes)


sampletype2  = "QC" 


QC_levels = [5]  # Change this to [5], [3, 5], [1, 3, 5, 7], etc.

QC_target_analytes = calculate_conc_targets(
    rawdata_copy,
    RF_target_analytes,
    EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
    Df_cal, sampletype2, debug_compound=None
)

# Filter to the specified QC level(s)
qc_indices = [level - 1 for level in QC_levels]  # Convert levels to indices

# Create expected concentrations dictionary with only the specified levels
expected_concs_qc = {}
for compound, conc_list in expected_concs_target_analytes.items():
    expected_concs_qc[compound] = [conc_list[idx] for idx in qc_indices]

calculated_recoveries_qc = calculate_recoveries(
    QC_target_analytes, 
    expected_concs_qc
)
#calculated_recoveries_QC = calculate_recoveries(QC_target_analytes, expected_concs_target_analytes)
# Save to CSV
calculated_recoveries_target_cal.to_csv('calculated_recoveries_target_cal.csv', index=False)
calculated_recoveries_qc.to_csv('calculated_recoveries_qc.csv', index=False)
validation_results_target_cal= validate_recoveries(calculated_recoveries_target_cal, IPR_recovery_rsd)
summary_cal = summarize_validation_results(validation_results_target_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

#for IPR, it is not necessarily failed if the below dataframe contains samples that do not pass. Rather, look at the output from mean_recoveries. 

failed_recoveries_cal = get_failed_recoveries(validation_results_target_cal)
#Only for IPR
IPR_target_cal = mean_recoveries(calculated_recoveries_target_cal, IPR_recovery_rsd)
display(IPR_target_cal)

#### Plotting cal line

In [ ]:
plot_calculated_vs_expected(conc_target_analytes_cal, expected_concs_target_analytes, analyte = "PFOA")

### 2.2.3 RSE results target analytes
We make two results, one with all calibration levels (7 for general compounds, 6 for FTS) - rse_results_cal_target, and one with the calibration level as excluded earlier for EIS compounds - rse_results_excluded_cal.

In [ ]:
%%capture
rse_results_cal_target, rse_results_excluded_cal = calculate_target_rse(conc_target_analytes_cal, expected_concs_target_analytes, p=2, level_to_exclude = excluded_cal_level )

In [ ]:
df_rse_target=df_rse_results(rse_results_cal_target)
display(df_rse_target)

In [ ]:
df_rse_target_excluded = df_rse_results(rse_results_excluded_cal) 
display(df_rse_target_excluded)

## 3 Samples 

### 3.1 Recovery EIS in Samples 
Here we compute the recovery of the EIS compounds within the samples analyzed. 

In [ ]:
# computation of EIS concentration and recovery rate
from solutions import NIS_stock, EIS_stock
from allowed_recoveries import EIS_NIS_compound_recovery_aqueous

DF_sample = 1 #dilution factor 
mass_bottle_filled = 499.8 #grams
mass_bottle_empty = 53.68 #grams
Ws = (mass_bottle_filled - mass_bottle_empty)/1000 #L
#Ws_cal = 0.25 #ml

# Create weights dictionary in grams
#soils
weights_dict = {
    'Sand500': 5.0,
    'Sand504': 5.04, 
    'Sand506': 5.06, 
    'Clay537': 5.37
}

#water 23-09
weights_dict = {
    'Alex_SPE50ml_1': 49.06/1000,#541.52-56.63,
    'Alex_SPE50ml_2': 48.64/1000, # 519.73-56.00, 
    'Alex_SPE50ml_3': 47.75/1000, #541.08-57.60, 
    'Alex_SPE50ml_4': 48.82/1000, #539.58-53.62,
    'Alex_SPE50ml_Blank':49.3/1000, # 529.43-57.80
}
rawdata_copy['Ws'] = rawdata_copy['Name'].map(weights_dict)

dilution_NIS_stock = 2#1:1 dilution
#dilution_NIS_stock_cal = 40 #dilution cal
#dilution_EIS_stock_cal = 40 #dilution cal
#spiked_amount_cal = 0.05 #ml
Df_cal = 1 #dilution factor
spiked_amount_NIS = 0.01 #ml
#spiked_amount = 0.01 #ml


dilution_EIS_stock = 1
spiked_amount_EIS =0.05 #ml
#look at this! 
calculated_conc_EIS, NIS_added, EIS_added = calculate_conc_EIS(rawdata_copy, eis_RFS, dilution_NIS_stock,EIS_NIS_analogs, DF_sample, NIS_stock, spiked_amount_NIS, spiked_amount_EIS)

#for run on 12-9-2025
expected_concs_EIS_doubled = {key: 2*value for key, value in expected_concs_EIS.items()}
calculated_recoveries_EIS = calculate_recoveries(calculated_conc_EIS, expected_concs_EIS_doubled)

validation_results_samples= validate_recoveries(calculated_recoveries_EIS, EIS_NIS_compound_recovery_aqueous)
summary_samples = summarize_validation_results(validation_results_samples)
print("Validation Summary:")
for key, value in summary_samples['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_samples = get_failed_recoveries(validation_results_samples)


## 3.2 QC samples
In this section we will check the results of the QC samples and assessing if they meet recovery limits. We have already computed EIS recovery for QC earlier. Here, we also compute the concentration for the target analytes in the QC samples.

In [ ]:
sample_QC = "QC"
QC_level = 5
conc_target_analytes_QC = calculate_conc_targets(rawdata_copy,
                           RF_target_analytes,
                           EIS_stock, dilution_EIS_stock_cal, spiked_amount_cal, Ws_cal,
                           Df_cal, sample_QC, debug_compound=None )

calculated_recoveries_EIS_QC = calculate_recoveries(conc_QC, expected_concs_EIS)
expected_concs_level =  filter_expected_concs_by_level(expected_concs_target_analytes,QC_level)
calculated_recoveries_target_QC = calculate_recoveries(conc_target_analytes_QC, expected_concs_level)

validation_results_target_QC= validate_recoveries(calculated_recoveries_target_QC, OPR_recovery_solid)

summary_cal = summarize_validation_results(validation_results_cal)
print("Validation Summary:")
for key, value in summary_cal['Overall'].items():
    print(f"{key}: {value}")

failed_recoveries_cal = get_failed_recoveries(validation_results_cal)
